In [18]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
print(tf. __version__) 

2.7.0


In [19]:
# import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

In [20]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [21]:
x_val  = x_train[50000:60000]
x_train = x_train[0:50000]
y_val  = y_train[50000:60000]
y_train = y_train[0:50000]

In [22]:
x_train = x_train.reshape(50000, 784)
x_val = x_val.reshape(10000, 784)
x_test = x_test.reshape(10000, 784)

In [23]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_val /= gray_scale
x_test /= gray_scale

In [24]:
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [25]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

In [26]:
def mlp(x):
    # hidden layer1
    w1 = tf.Variable(tf.random_uniform([784,256]))
    b1 = tf.Variable(tf.zeros([256]))
    h1 = tf.nn.relu(tf.matmul(x, w1) + b1)
    # hidden layer2
    w2 = tf.Variable(tf.random_uniform([256,128]))
    b2 = tf.Variable(tf.zeros([128]))
    h2 = tf.nn.relu(tf.matmul(h1, w2) + b2)
    # output layer
    w3 = tf.Variable(tf.random_uniform([128,10]))
    b3 = tf.Variable(tf.zeros([10]))
    logits= tf.matmul(h2, w3) + b3
    
    return logits

In [27]:
logits = mlp(x)

In [28]:
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
logits=logits, labels=y))

train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss_op)

In [29]:
# initialize
init = tf.global_variables_initializer()

# train hyperparameters
epoch_cnt = 30
batch_size = 1000
iteration = len(x_train) // batch_size

In [30]:
# Start training
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    for epoch in range(epoch_cnt):
        avg_loss = 0.
        start = 0; end = batch_size
        
        for i in range(iteration):
            _, loss = sess.run([train_op, loss_op], 
                               feed_dict={x: x_train[start: end], y: y_train[start: end]})
            start += batch_size; end += batch_size
            # Compute average loss
            avg_loss += loss / iteration
            
        # Validate model
        preds = tf.nn.softmax(logits)  # Apply softmax to logits
        correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        cur_val_acc = accuracy.eval({x: x_val, y: y_val})
        print("epoch: "+str(epoch)+", validation accuracy: " 
              + str(cur_val_acc) +', loss: '+str(avg_loss))
    
    # Test model
    preds = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("[Test Accuracy] :", accuracy.eval({x: x_test, y: y_test}))

epoch: 0, validation accuracy: 0.0967, loss: 11570.591376953122
epoch: 1, validation accuracy: 0.7818, loss: 326.71033813476566
epoch: 2, validation accuracy: 0.8719, loss: 12.549036426544188
epoch: 3, validation accuracy: 0.8895, loss: 7.307183084487915
epoch: 4, validation accuracy: 0.8957, loss: 5.768688158988954
epoch: 5, validation accuracy: 0.9018, loss: 4.743809337615968
epoch: 6, validation accuracy: 0.9014, loss: 4.075223827362059
epoch: 7, validation accuracy: 0.9036, loss: 3.659175968170165
epoch: 8, validation accuracy: 0.8932, loss: 3.1618099355697638
epoch: 9, validation accuracy: 0.894, loss: 3.30657993555069
epoch: 10, validation accuracy: 0.9057, loss: 2.922093641757965
epoch: 11, validation accuracy: 0.9005, loss: 2.6804901957511906
epoch: 12, validation accuracy: 0.886, loss: 4.08508975982666
epoch: 13, validation accuracy: 0.9064, loss: 2.8736781620979315
epoch: 14, validation accuracy: 0.8898, loss: 2.567079949378967
epoch: 15, validation accuracy: 0.8993, loss: 4.